# 楽曲のコード進行を取得したい。

## Songleを用いたやり方  

https://songle.jp/ にはAPIが公開されている  
http://widget.songle.jp/docs/v1 を読むと、  
http://widget.songle.jp/api/v1/song/chord.json?url=www.youtube.com/watch?v=PqJNc9KVIZE  
のようにすればコード進行のjsonを取得できるようだ。

REST API なのでHTTPメソッドでアクセスすればよく、pythonからはrequestsでデータを取得可能。  
https://note.nkmk.me/python-requests-web-api/

In [2]:
import requests

In [3]:
song_url = "www.youtube.com/watch?v=LtSNzPyo0lA"
# ぼくらはみんな意味不明 ピノキオピー

r_get = requests.get("http://widget.songle.jp/api/v1/song/chord.json?url=" + song_url)

print(r_get.status_code)

200


In [4]:
r_get.headers

{'Server': 'nginx', 'Date': 'Thu, 20 Dec 2018 14:21:19 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Status': '200 OK', 'Cache-Control': 'max-age=0, private, must-revalidate', 'Access-Control-Allow-Origin': '*', 'ETag': 'W/"d78bac811763b859e90b56a77a7894c4"', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Runtime': '0.040452', 'X-Request-Id': 'cef4f0f5-01b6-4b2d-a2d7-95c672205a06', 'Content-Encoding': 'gzip'}

In [5]:
p = r_get.json()['chords'] 
# キー'chords'に対応する値を取得する

# 作戦会議

「ある曲の「ある部分の」転調したときのコード構成が他の曲の「ある部分の」コード構成と類似している」ことが知りたい  

・曲の区間を選び出す 方法は?→ 繰り返し構造がアノテートされているから、そこを拾えば行けそうだ感がある  
・そこのコードの音程を分析する  
    ・パース 原音とコード名 / N  
    ・音程の導出  
    ・分数コード(MVP段階では捨てても良い)  
    
・テンポ  
　　・基準となる数値を見つける  
  　・それぞれのdurationを基準値で割ってクオンタイズ  
    ・同時に、BPMを推定

・転調 は総当たりで行けるだろう、音程変化は楽

・コード構成の類似  durationが一致とは限らないので難しいが、基本的には構成音の共通がどれくらい多いかで見れば良さそう

# 作戦会議2

いや……一旦2曲の類似を確認するくらいならそんなに要らないんじゃないか?

・サビ同士を決め打ちで区間抽出して、決め打ちで転調して、コード構成の類似を見れば良い!

In [11]:
# 約62.640秒から約17.200秒間のコードを判別する
import json
stt = json.dumps(p)

type(stt)

str

In [31]:
import pandas as pd

df00 = pd.read_json(stt)

# durationを1060で割ってみる
unit_ms = 1060
df00["duration"] = (df00["duration"]/ unit_ms).round().astype("int64")

In [48]:

df00.head()

,duration,index,name,start
0,3,0,N,0
1,2,1,Abm7,3283
2,1,2,C#m7,5411
3,1,3,F#,6467
4,2,4,Abm7,7533


In [49]:
import re

def chord_to_pitch(chord_name):
    # 文字列を受け取ると音の配列を返す
    # 0がC, 1がC#, ..., 11がB
    # chrod_nameが"N"ならば空配列
    if chord_name == "N":
        return {}
    # 一旦分数コードを考慮外にしたいので、 /以降を切り捨てる
    chord_name = re.search("^[^/]+", chord_name)
    # ^ 先頭、 ^/ スラッシュ以外、 + 一文字以上繰り返し
    # searchで返ってくるのはmatchオブジェクトなので、その中身をgroupで取得
    chord_name = chord_name.group()
